In [27]:
import random
random.seed(0)
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [28]:
np.random.seed(0)
import tensorflow as tf

In [29]:
tf.random.set_seed(0)

In [30]:
import os
import json
from zipfile import ZipFile
from PIL import Image

In [31]:
import numpy as np
import matplotlib.pyplot as plt

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers,models

In [33]:
!pip install kaggle

In [34]:
kaggle_crendentials=json.load(open('kaggle (1).json'))

# New Section

In [35]:
os.environ['KAGGLE_USERNAME']=kaggle_crendentials['username']
os.environ['KAGGLE_KEY']=kaggle_crendentials['key']

In [36]:
!kaggle datasets download -d emmarex/plantdisease

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
plantdisease.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip plantdisease.zip

Archive:  plantdisease.zip
replace PlantVillage/Pepper__bell___Bacterial_spot/0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot 8964.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
with ZipFile('plantdisease.zip','r')as zip_ref:
  zip_ref.extractall()

In [ ]:
ls

In [ ]:
print(os.listdir("plantvillage"))

In [ ]:
print(len(os.listdir("PlantVillage/Tomato_Early_blight")))

In [ ]:
base_dir="plantvillage/PlantVillage/Pepper__bell___Bacterial_spot"

In [ ]:
import matplotlib.image as mpimg
img_path="/content/plantvillage/PlantVillage/Pepper__bell___Bacterial_spot/0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot 8964.JPG"
img=mpimg.imread(img_path)
print(img.shape)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
img_size=224
batch_size=32

In [ ]:
data_gen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
train_generator=data_gen.flow_from_directory(
    directory='/content/plantvillage/PlantVillage',
    target_size=(img_size,img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'

)

In [ ]:
validation_generator=data_gen.flow_from_directory(
    directory='/content/plantvillage/PlantVillage',
    target_size=(img_size,img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'

)

In [ ]:
from tensorflow.keras import layers, models

num_classes = train_generator.num_classes
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Flatten())  # "flatten" with capital F

model.add(layers.Dense(256, activation='relu'))

# Use 'num_classes' directly, not 'train_generator'
model.add(layers.Dense(num_classes, activation='softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//batch_size
)

In [ ]:
print("Evaluating the model...")
val_loss, val_acc = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("model accuracy")
plt.xlabel('Accuracy')
plt.ylabel('Epoch')
plt.legend('train','test',loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend({'train','test'},loc='upper left')
plt.show()


In [ ]:
def load_and_preprocess_image(image_path,target_size=[224,224]):
  img=Image.open(image_path)
  img=img.resize(target_size)
  img_array=np.array(img)
  img_array=np.expand_dims(img_array,axis=0)
  img_array=img_array.astype('float32')/255
  return img_array
def predict_image_class(model,image_path,class_indices):
  preprocessed_img=load_and_preprocess_image(image_path)
  predictions=model.predict(preprocessed_img)
  predicted_class_index=np.argmax(predictions,axis=1)[0]
  predicted_class_name=class_indices[predicted_class_index]
  return predicted_class_name

In [ ]:
class_indices = {v: k for k, v in train_generator.class_indices.items()}


In [ ]:
image_path='/content/plantvillage/PlantVillage/Tomato_Bacterial_spot/00416648-be6e-4bd4-bc8d-82f43f8a7240___GCREC_Bact.Sp 3110.JPG'
image_path='/content/plantvillage/PlantVillage/Tomato_Early_blight/0012b9d2-2130-4a06-4a834-b1f3af34f57e___RS_Erly.B 8389.JPG'
image_path='/content/plantvillage/PlantVillage/Tomato_Late_blight/0003faa8-4b27-4c65-bf42-6d9e352ca1a5___RS_Late.B 4946.JPG'
predicted_class_name=predict_image_class(model,image_path,class_indices)
print(f"predicted class ",predicted_class_name)

In [ ]:
model.save('/content/drive/My Drive/plant_disease_model.h5')


In [ ]:
class_indices